In [1]:
from gensim.models.keyedvectors import KeyedVectors
import numpy as np

import codecs
import argparse
import csv
import random

from collections import defaultdict

# very useful feature used to reload python modules
from importlib import reload

# import module that loads data, tokenises the tuples, initialises the embeddings matrix
import crim_data

import multiprojection_model
# contains code to evaluate according to semeval2018 metrics
import semeval_eval
import crim_evaluator

from sklearn.model_selection import KFold

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# initialise embeddings and normalise to unit-norm
#model = KeyedVectors.load_word2vec_format('embeddings/GoogleNews-vectors-negative300.bin', binary=True)
#model = KeyedVectors.load_word2vec_format('embeddings/glove.42B.300d.txt', binary=False)
model = KeyedVectors.load_word2vec_format('embeddings/wiki-news-300d-1M.vec', binary=False)

#model.save_word2vec_format('embeddings/GoogleNews-vectors-negative300.txt', binary=False)
model.init_sims(replace=True)

In [3]:
# import Ustalov et al. prepare module
import prepare

sub_train = prepare.read_subsumptions('subsumptions-train.txt')
sub_test = prepare.read_subsumptions('subsumptions-test.txt')
sub_validation = prepare.read_subsumptions('subsumptions-validation.txt')
synonyms = prepare.read_synonyms('synonyms.txt')  

# remove vocab term having no vector in embeddings
def get_terms_having_vectors(w2v, dataset):
    return [(q,h) for q, h in dataset if q in w2v and h in w2v]

sub_train = get_terms_having_vectors(model, sub_train)
sub_test = get_terms_having_vectors(model, sub_test)
sub_validation = get_terms_having_vectors(model, sub_validation)
synonyms = prepare.get_synonymys_having_vectors(synonyms, model)
synonyms.default_factory = None

# create hypernym dictionary
hyper_dict = defaultdict(list)
for x, y in sub_train + sub_test + sub_validation:        
    hyper_dict[x].append(y)
    
hyper_dict.default_factory = None

print ("Total number of tuples in entire set: %d" % (len([x for (x,y) in sub_train + sub_test + sub_validation])))
print ("Unique hyponyms in set: %d" % (len(set([x for (x,y) in sub_train + sub_test + sub_validation]))))

Total number of tuples in entire set: 7377
Unique hyponyms in set: 2247


In [4]:
args = {'w2v':model,
        'train':sub_train, 'test':sub_test, 'validation':sub_validation, 'synonyms':synonyms, 
        'limited_vocab_n': 250000
       }
data = crim_data.CrimData(args)

Creating limited vocabulary of 250000
Dataset vocabulary size is 3232
Truncated vocab length is 250000
Vocab size is 250000 words
Initialising negative sampler
Tokenising all dataset tuples
Creating embeddings matrix
Done!


In [5]:
# convert full dataset to array
all_data_tokens = np.asarray(data.all_data_token)

kf = KFold(n_splits=5)
kf.get_n_splits(all_data_tokens[:,0])

# split data into 5 different train-test folds
train_data_split = []
test_data_split = []
for k in kf.split(all_data_tokens[:,0]):    
    k_train_split = all_data_tokens[k[0]]
    k_test_split = all_data_tokens[k[1]]
    
    train_data_split.append(k_train_split)
    test_data_split.append(k_test_split)

# output training-test split sizes    
for tr, te in zip(train_data_split, test_data_split):
    print ("Training tuples: %d; test tuples: %d" % (len(tr), len(te)))

Training tuples: 5901; test tuples: 1476
Training tuples: 5901; test tuples: 1476
Training tuples: 5902; test tuples: 1475
Training tuples: 5902; test tuples: 1475
Training tuples: 5902; test tuples: 1475


In [6]:
def train_and_evaluate_1_fold(hyp_model, train_split, test_split):    
    
    # fit model
    # the test split is only used to measure the test loss
    hyp_model.fit(train_split, test_split)    
    # this step should not be required since the model is dynamically linked to the evaluator
    hyp_model.evaluator.set_model(hyp_model.model)
    # generates predictions according to trained model
    predictions = hyp_model.evaluator.predict(test_split)
    # this converts the tokens back to words for evaluation
    test_tuples = data.token_to_words(test_split)
    # here we have a scorer that will mark our effort according to this particular test split
    scorer = semeval_eval.HypernymEvaluation(test_tuples)
    # get scores
    score_names, all_scores = scorer.get_evaluation_scores(predictions)
    # initialise scores (MRR, MAP, ...)
    scores = {s:0.0 for s in score_names }
    for k in range(len(score_names)):    
        scores[score_names[k]] = float('%.5f' % (sum([score_list[k] for score_list in all_scores]) / len(all_scores)))    

    return scores
    

In [7]:
from itertools import product

# standard model parameters - we won't be changing these
args['data'] = data
args['epochs'] = 15
args['batch_size']= 32
args['synonym_sample_n']= 5
args['phi_k'] = 1
args['lambda_c'] = 0.
args['negative_sample_n'] = 10

# generate parameter combinations
_clusters = [1, 5, 10]
_lambda_c = [0, 0.1, 1]
_neg_count = [1, 5, 10]

parameters = [_clusters, _lambda_c, _neg_count]

param_list = list(product(*parameters))

# initialise hypernymy discovery model which we will reuse by resetting the model with new args
hyp_model = multiprojection_model.MultiProjModel(args)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
# initialise final_scores dictionary
final_scores = {k:defaultdict(list) for k in param_list}

for _param in param_list:
    print ("Running test with following parameters: phi_k: %d; lambda_c: %0.2f; neg_count: %d" \
           % (_param[0], _param[1], _param[2]))

    args['phi_k'] = _param[0]
    args['lambda_c'] = _param[1]
    args['negative_sample_n'] = _param[2]    
    
    # iterate over every split to get score distribution
    for idx, td in enumerate(train_data_split):              
        hyp_model.reset_model(args=args)
        
        scores = train_and_evaluate_1_fold(hyp_model, td, test_data_split[idx])
        for s, v  in scores.items():
            final_scores[_param][s].append(v)
    print ("-"*30)

Running test with following parameters: phi_k: 1; lambda_c: 0.00; neg_count: 1
Fitting model with following parameters: batch_size=32; phi_k=1; lambda_c=0.00; epochs=15; negative_count=1; synonym_count=5
Instructions for updating:
Use tf.cast instead.
Epoch: 1; Training Loss: 0.67537; Test Loss: 0.65366
Epoch: 2; Training Loss: 0.65613; Test Loss: 0.62789
Epoch: 3; Training Loss: 0.63816; Test Loss: 0.60199
Epoch: 4; Training Loss: 0.62051; Test Loss: 0.57895
Epoch: 5; Training Loss: 0.60194; Test Loss: 0.55422
Epoch: 6; Training Loss: 0.58548; Test Loss: 0.53395
Epoch: 7; Training Loss: 0.56923; Test Loss: 0.51206
Epoch: 8; Training Loss: 0.55311; Test Loss: 0.49182
Epoch: 9; Training Loss: 0.53765; Test Loss: 0.47205
Epoch: 10; Training Loss: 0.52256; Test Loss: 0.45305
Epoch: 11; Training Loss: 0.50684; Test Loss: 0.43639
Epoch: 12; Training Loss: 0.49405; Test Loss: 0.41891
Epoch: 13; Training Loss: 0.48107; Test Loss: 0.40615
Epoch: 14; Training Loss: 0.46626; Test Loss: 0.38739
E

Epoch: 1; Training Loss: 0.65404; Test Loss: 0.60546
Epoch: 2; Training Loss: 0.58332; Test Loss: 0.53480
Epoch: 3; Training Loss: 0.52647; Test Loss: 0.47908
Epoch: 4; Training Loss: 0.48100; Test Loss: 0.43532
Epoch: 5; Training Loss: 0.44479; Test Loss: 0.40212
Epoch: 6; Training Loss: 0.41746; Test Loss: 0.37580
Epoch: 7; Training Loss: 0.39496; Test Loss: 0.35532
Epoch: 8; Training Loss: 0.37822; Test Loss: 0.33854
Epoch: 9; Training Loss: 0.36586; Test Loss: 0.32510
Epoch: 10; Training Loss: 0.35650; Test Loss: 0.31268
Epoch: 11; Training Loss: 0.34719; Test Loss: 0.30165
Epoch: 12; Training Loss: 0.33797; Test Loss: 0.29031
Epoch: 13; Training Loss: 0.32974; Test Loss: 0.27920
Epoch: 14; Training Loss: 0.32129; Test Loss: 0.26875
Epoch: 15; Training Loss: 0.31265; Test Loss: 0.25900
Done 100
Done 200
Done 300
Done 400
Fitting model with following parameters: batch_size=32; phi_k=1; lambda_c=0.00; epochs=15; negative_count=5; synonym_count=5
Epoch: 1; Training Loss: 0.64901; Test

Epoch: 7; Training Loss: 0.58430; Test Loss: 0.53290
Epoch: 8; Training Loss: 0.56884; Test Loss: 0.51452
Epoch: 9; Training Loss: 0.55305; Test Loss: 0.49436
Epoch: 10; Training Loss: 0.53759; Test Loss: 0.47168
Epoch: 11; Training Loss: 0.52281; Test Loss: 0.45773
Epoch: 12; Training Loss: 0.50986; Test Loss: 0.43941
Epoch: 13; Training Loss: 0.49546; Test Loss: 0.42296
Epoch: 14; Training Loss: 0.48163; Test Loss: 0.40783
Epoch: 15; Training Loss: 0.46788; Test Loss: 0.39082
Done 100
Done 200
Done 300
Done 400
Fitting model with following parameters: batch_size=32; phi_k=1; lambda_c=0.10; epochs=15; negative_count=1; synonym_count=5
Epoch: 1; Training Loss: 0.68124; Test Loss: 0.66202
Epoch: 2; Training Loss: 0.66342; Test Loss: 0.63776
Epoch: 3; Training Loss: 0.64585; Test Loss: 0.61429
Epoch: 4; Training Loss: 0.62887; Test Loss: 0.59081
Epoch: 5; Training Loss: 0.61157; Test Loss: 0.57023
Epoch: 6; Training Loss: 0.59432; Test Loss: 0.54852
Epoch: 7; Training Loss: 0.57762; Test

Epoch: 15; Training Loss: 0.31937; Test Loss: 0.27079
Done 100
Done 200
Done 300
Done 400
------------------------------
Running test with following parameters: phi_k: 1; lambda_c: 0.10; neg_count: 10
Fitting model with following parameters: batch_size=32; phi_k=1; lambda_c=0.10; epochs=15; negative_count=10; synonym_count=5
Epoch: 1; Training Loss: 0.64787; Test Loss: 0.58874
Epoch: 2; Training Loss: 0.56214; Test Loss: 0.50053
Epoch: 3; Training Loss: 0.49013; Test Loss: 0.42965
Epoch: 4; Training Loss: 0.43039; Test Loss: 0.37358
Epoch: 5; Training Loss: 0.38276; Test Loss: 0.33004
Epoch: 6; Training Loss: 0.34469; Test Loss: 0.29697
Epoch: 7; Training Loss: 0.31489; Test Loss: 0.27225
Epoch: 8; Training Loss: 0.29207; Test Loss: 0.25381
Epoch: 9; Training Loss: 0.27386; Test Loss: 0.23979
Epoch: 10; Training Loss: 0.25939; Test Loss: 0.22875
Epoch: 11; Training Loss: 0.24928; Test Loss: 0.22105
Epoch: 12; Training Loss: 0.24165; Test Loss: 0.21495
Epoch: 13; Training Loss: 0.23627;

Epoch: 3; Training Loss: 0.65976; Test Loss: 0.63511
Epoch: 4; Training Loss: 0.64460; Test Loss: 0.61379
Epoch: 5; Training Loss: 0.63000; Test Loss: 0.59359
Epoch: 6; Training Loss: 0.61523; Test Loss: 0.57400
Epoch: 7; Training Loss: 0.59964; Test Loss: 0.55610
Epoch: 8; Training Loss: 0.58540; Test Loss: 0.53496
Epoch: 9; Training Loss: 0.56993; Test Loss: 0.51742
Epoch: 10; Training Loss: 0.55782; Test Loss: 0.49903
Epoch: 11; Training Loss: 0.54474; Test Loss: 0.48627
Epoch: 12; Training Loss: 0.53134; Test Loss: 0.46878
Epoch: 13; Training Loss: 0.51852; Test Loss: 0.45412
Epoch: 14; Training Loss: 0.50822; Test Loss: 0.43880
Epoch: 15; Training Loss: 0.49470; Test Loss: 0.42511
Done 100
Done 200
Done 300
Done 400
Fitting model with following parameters: batch_size=32; phi_k=1; lambda_c=1.00; epochs=15; negative_count=1; synonym_count=5
Epoch: 1; Training Loss: 0.69204; Test Loss: 0.67819
Epoch: 2; Training Loss: 0.67698; Test Loss: 0.65731
Epoch: 3; Training Loss: 0.66190; Test

...

Epoch: 1; Training Loss: 0.51548; Test Loss: 0.31326
Epoch: 2; Training Loss: 0.34247; Test Loss: 0.19616
Epoch: 3; Training Loss: 0.25060; Test Loss: 0.13577
Epoch: 4; Training Loss: 0.19348; Test Loss: 0.09965
Epoch: 5; Training Loss: 0.15331; Test Loss: 0.07417
Epoch: 6; Training Loss: 0.12495; Test Loss: 0.05912
Epoch: 7; Training Loss: 0.10214; Test Loss: 0.05012
Epoch: 8; Training Loss: 0.08674; Test Loss: 0.04112
Epoch: 9; Training Loss: 0.07399; Test Loss: 0.03764
Epoch: 10; Training Loss: 0.06369; Test Loss: 0.03363
Epoch: 11; Training Loss: 0.05560; Test Loss: 0.03022
Epoch: 12; Training Loss: 0.04925; Test Loss: 0.02740
Epoch: 13; Training Loss: 0.04455; Test Loss: 0.02576
Epoch: 14; Training Loss: 0.03923; Test Loss: 0.02494
Epoch: 15; Training Loss: 0.03562; Test Loss: 0.02452
Done 100
Done 200
Done 300
Done 400
Fitting model with following parameters: batch_size=32; phi_k=10; lambda_c=0.00; epochs=15; negative_count=10; synonym_count=5
Epoch: 1; Training Loss: 0.54133; Te

Epoch: 9; Training Loss: 0.09609; Test Loss: 0.06428
Epoch: 10; Training Loss: 0.08500; Test Loss: 0.06067
Epoch: 11; Training Loss: 0.07447; Test Loss: 0.06379
Epoch: 12; Training Loss: 0.06602; Test Loss: 0.04749
Epoch: 13; Training Loss: 0.06068; Test Loss: 0.04123
Epoch: 14; Training Loss: 0.05752; Test Loss: 0.04034
Epoch: 15; Training Loss: 0.05286; Test Loss: 0.04328
Done 100
Done 200
Done 300
Done 400
Fitting model with following parameters: batch_size=32; phi_k=10; lambda_c=0.10; epochs=15; negative_count=1; synonym_count=5
Epoch: 1; Training Loss: 0.59786; Test Loss: 0.45578
Epoch: 2; Training Loss: 0.45432; Test Loss: 0.31151
Epoch: 3; Training Loss: 0.34736; Test Loss: 0.21686
Epoch: 4; Training Loss: 0.26687; Test Loss: 0.15630
Epoch: 5; Training Loss: 0.21123; Test Loss: 0.12346
Epoch: 6; Training Loss: 0.17249; Test Loss: 0.09716
Epoch: 7; Training Loss: 0.13730; Test Loss: 0.08666
Epoch: 8; Training Loss: 0.11852; Test Loss: 0.07046
Epoch: 9; Training Loss: 0.09854; Tes

Epoch: 14; Training Loss: 0.04042; Test Loss: 0.02514
Epoch: 15; Training Loss: 0.03542; Test Loss: 0.02419
Done 100
Done 200
Done 300
Done 400
Fitting model with following parameters: batch_size=32; phi_k=10; lambda_c=0.10; epochs=15; negative_count=10; synonym_count=5
Epoch: 1; Training Loss: 0.52713; Test Loss: 0.32469
Epoch: 2; Training Loss: 0.34779; Test Loss: 0.20409
Epoch: 3; Training Loss: 0.25463; Test Loss: 0.14039
Epoch: 4; Training Loss: 0.19628; Test Loss: 0.10225
Epoch: 5; Training Loss: 0.15593; Test Loss: 0.07656
Epoch: 6; Training Loss: 0.12616; Test Loss: 0.05999
Epoch: 7; Training Loss: 0.10310; Test Loss: 0.05165
Epoch: 8; Training Loss: 0.08798; Test Loss: 0.04429
Epoch: 9; Training Loss: 0.07367; Test Loss: 0.03667
Epoch: 10; Training Loss: 0.06411; Test Loss: 0.03485
Epoch: 11; Training Loss: 0.05531; Test Loss: 0.03001
Epoch: 12; Training Loss: 0.04960; Test Loss: 0.02851
Epoch: 13; Training Loss: 0.04380; Test Loss: 0.02702
Epoch: 14; Training Loss: 0.03986; T

Epoch: 1; Training Loss: 0.55865; Test Loss: 0.37528
Epoch: 2; Training Loss: 0.39298; Test Loss: 0.24730
Epoch: 3; Training Loss: 0.29764; Test Loss: 0.16995
Epoch: 4; Training Loss: 0.23041; Test Loss: 0.12348
Epoch: 5; Training Loss: 0.18360; Test Loss: 0.09519
Epoch: 6; Training Loss: 0.14744; Test Loss: 0.07687
Epoch: 7; Training Loss: 0.12334; Test Loss: 0.06241
Epoch: 8; Training Loss: 0.10183; Test Loss: 0.05831
Epoch: 9; Training Loss: 0.08681; Test Loss: 0.04421
Epoch: 10; Training Loss: 0.07671; Test Loss: 0.04172
Epoch: 11; Training Loss: 0.06682; Test Loss: 0.04093
Epoch: 12; Training Loss: 0.05856; Test Loss: 0.03636
Epoch: 13; Training Loss: 0.05290; Test Loss: 0.03387
Epoch: 14; Training Loss: 0.04700; Test Loss: 0.03565
Epoch: 15; Training Loss: 0.04250; Test Loss: 0.03107
Done 100
Done 200
Done 300
Done 400
Fitting model with following parameters: batch_size=32; phi_k=10; lambda_c=1.00; epochs=15; negative_count=5; synonym_count=5
Epoch: 1; Training Loss: 0.57169; Tes

Epoch: 9; Training Loss: 0.07595; Test Loss: 0.03696
Epoch: 10; Training Loss: 0.06578; Test Loss: 0.03128
Epoch: 11; Training Loss: 0.05754; Test Loss: 0.02885
Epoch: 12; Training Loss: 0.05058; Test Loss: 0.02721
Epoch: 13; Training Loss: 0.04521; Test Loss: 0.02642
Epoch: 14; Training Loss: 0.04058; Test Loss: 0.02805
Epoch: 15; Training Loss: 0.03717; Test Loss: 0.02415
Done 100
Done 200
Done 300
Done 400
Fitting model with following parameters: batch_size=32; phi_k=10; lambda_c=1.00; epochs=15; negative_count=10; synonym_count=5
Epoch: 1; Training Loss: 0.53529; Test Loss: 0.32948
Epoch: 2; Training Loss: 0.35234; Test Loss: 0.20430
Epoch: 3; Training Loss: 0.25684; Test Loss: 0.13923
Epoch: 4; Training Loss: 0.19953; Test Loss: 0.10088
Epoch: 5; Training Loss: 0.15696; Test Loss: 0.07619
Epoch: 6; Training Loss: 0.12807; Test Loss: 0.05902
Epoch: 7; Training Loss: 0.10592; Test Loss: 0.04859
Epoch: 8; Training Loss: 0.08782; Test Loss: 0.04068
Epoch: 9; Training Loss: 0.07562; Te

In [9]:
final_scores

{(1, 0, 1): defaultdict(list,
             {'MRR': [0.53408, 0.52189, 0.52402, 0.53716, 0.52682],
              'MAP': [0.35661, 0.35583, 0.36184, 0.35876, 0.36184],
              'P@1': [0.51386, 0.49353, 0.50227, 0.51522, 0.50334],
              'P@5': [0.34542, 0.34828, 0.34841, 0.34856, 0.35148],
              'P@10': [0.32982, 0.32958, 0.33627, 0.32854, 0.33504]}),
 (1, 0, 5): defaultdict(list,
             {'MRR': [0.53396, 0.52407, 0.52383, 0.53747, 0.53255],
              'MAP': [0.36177, 0.35579, 0.36238, 0.36598, 0.36429],
              'P@1': [0.50959, 0.48922, 0.50227, 0.51288, 0.5078],
              'P@5': [0.35299, 0.34853, 0.35197, 0.35461, 0.35612],
              'P@10': [0.33688, 0.32936, 0.33606, 0.33696, 0.3368]}),
 (1, 0, 10): defaultdict(list,
             {'MRR': [0.53397, 0.52863, 0.52576, 0.53774, 0.53313],
              'MAP': [0.36001, 0.35801, 0.35885, 0.36343, 0.368],
              'P@1': [0.51173, 0.50431, 0.50227, 0.51991, 0.51002],
              'P@5': [0

{'MRR': 0.48189,
 'MAP': 0.28752,
 'P@1': 0.46398,
 'P@5': 0.27521,
 'P@10': 0.25954}
 
Fitting model with following parameters: batch_size=32; phi_k=10; lambda_c=1.00; epochs=15; negative_count=10; synonym_count=5
 {'MRR': 0.45751,
 'MAP': 0.30459,
 'P@1': 0.39407,
 'P@5': 0.30215,
 'P@10': 0.28868}

{'MRR': 0.26624,
 'MAP': 0.17138,
 'P@1': 0.19492,
 'P@5': 0.1721,
 'P@10': 0.16572}
 
{'MRR': 0.43478,
 'MAP': 0.29167,
 'P@1': 0.38136,
 'P@5': 0.28898,
 'P@10': 0.27513}
 
{'MRR': 0.52221,
 'MAP': 0.3812,
 'P@1': 0.49467,
 'P@5': 0.37676,
 'P@10': 0.36168}

# Scratch Pad

In [ ]:
reload(crim_data)
reload(multiprojection_model)
reload(crim_evaluator)

In [ ]:
predictions

In [ ]:
# get queries from tuples
#hyp_model.evaluator.predict_word('mare')
predictions = hyp_model.evaluator.predict(test_data_split[0])

In [ ]:
test_tuples = data.token_to_words(test_data_split[0])
scorer = semeval_eval.HypernymEvaluation(test_tuples)

# get scores
score_names, all_scores = scorer.get_evaluation_scores(predictions)

scores = {s:0.0 for s in score_names }

for k in range(len(score_names)):    
    scores[score_names[k]] = float('%.5f' % (sum([score_list[k] for score_list in all_scores]) / len(all_scores)))    

scores

In [ ]:
from itertools import product
# create score dictionary
_clusters = [1, 5, 10]
_lambda_c = [0, 0.1, 1]
_neg_count = [1, 5, 10]

parameters = [_clusters, _lambda_c, _neg_count]

param_list = list(product(*parameters))
final_scores = {k:defaultdict(list) for k in param_list}